In [3]:
import requests
import os
import tarfile
import tensorflow as tf
import shutil


2024-05-21 06:36:51.297866: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS, tf.float16]

# Set the input and output tensors to the converter
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]

Failed to initialize NVML: GPU access blocked by the operating system
Failed to properly shut down NVML: GPU access blocked by the operating system



In [4]:
models = [
    "http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_1024x1024_coco17_tpu-32.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_1024x1024_kpts_coco17_tpu-32.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v1_fpn_512x512_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v1_fpn_512x512_kpts_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet101_v1_fpn_512x512_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v2_512x512_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v2_512x512_kpts_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_kpts.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d2_coco17_tpu-32.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d3_coco17_tpu-32.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d4_coco17_tpu-32.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d5_coco17_tpu-32.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d6_coco17_tpu-32.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d7_coco17_tpu-32.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_640x640_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_800x1333_coco17_gpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz",
    "http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz",
]
print(len(models))
pattern = r'(\d+)x(\d+)'


40


In [13]:
import re

def download_model(model_url, path):
    """
    Download a model from the given URL and save it to the specified path.

    Args:
    model_url (str): The URL of the model to download.
    path (str): The path where the model will be saved.
    """
    # Check if the directory exists, if not, create it
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))

    # Send a GET request to download the model
    print()
    response = requests.get(model_url, stream=True)
    with open(path, 'wb') as f:
        # Write the content of the response to the file
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)

    print(f"Model downloaded successfully at {path}")
    return path
def extract_tar(path):
    """
    Extract a tar file to the specified path.

    Args:
    path (str): Path to the tar file.
    """
    with tarfile.open(path, "r") as tar:
        tar.extractall()

    print(f"Contents of {path} extracted to {path[:-7]}")
    return path[:-7]

def saved_model_2_tflite(saved_model_dir, tflite_output_path):
    converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
    tflite_model = converter.convert()
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS, tf.float16]
    # Set the input and output tensors to the converter
    converter.experimental_new_converter = True
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    with open(tflite_output_path + ".tflite", 'wb') as f:
        f.write(tflite_model)

In [15]:
for i in range(19,24):
    model_url = models[i]
    model_tar_file = model_url.split("/")[-1]
    
    downloaded_model_path = download_model(model_url, "./" + model_tar_file)
    
    extract_path = extract_tar(downloaded_model_path)
    
    if os.path.isfile(downloaded_model_path):
        os.remove(downloaded_model_path)
    
    saved_model_2_tflite(extract_path + "/saved_model", "tflite_models/" + extract_path[2:])
    
    try:
        shutil.rmtree(extract_path)
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))


Model downloaded successfully at ./ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
Contents of ./ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz extracted to ./ssd_mobilenet_v2_320x320_coco17_tpu-8


W0000 00:00:1716275323.123252      27 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1716275323.123345      27 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-05-21 07:08:43.123607: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model
2024-05-21 07:08:43.203982: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-05-21 07:08:43.204029: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: ./ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model
2024-05-21 07:08:43.849693: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-05-21 07:08:44.608737: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: ./ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model
2024-05-21 07:08:45.058930: I tensorflow/cc/saved_model/loader.cc:317] SavedModel load for ta


Model downloaded successfully at ./ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz
Contents of ./ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz extracted to ./ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8


W0000 00:00:1716275348.550843      27 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1716275348.550895      27 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-05-21 07:09:08.551090: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/saved_model
2024-05-21 07:09:08.617229: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-05-21 07:09:08.617265: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: ./ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/saved_model
2024-05-21 07:09:09.209448: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-05-21 07:09:10.300314: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: ./ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/saved_model
2024-05-21 07:09:11.038071: I tensorflow/cc/saved_model/loader.cc:317] SavedModel


Model downloaded successfully at ./ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Contents of ./ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz extracted to ./ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8


W0000 00:00:1716275373.824252      27 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1716275373.824300      27 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-05-21 07:09:33.824485: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model
2024-05-21 07:09:33.894811: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-05-21 07:09:33.894845: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: ./ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model
2024-05-21 07:09:34.555573: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-05-21 07:09:35.482546: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: ./ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model
2024-05-21 07:09:35.974697: I tensorflow/cc/saved_model/loader.cc:317


Model downloaded successfully at ./ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
Contents of ./ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz extracted to ./ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8


W0000 00:00:1716275394.230033      27 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1716275394.230080      27 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-05-21 07:09:54.230266: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model
2024-05-21 07:09:54.305965: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-05-21 07:09:54.306000: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: ./ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model
2024-05-21 07:09:54.991606: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-05-21 07:09:55.889923: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: ./ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model
2024-05-21 07:09:56.408579: I tensorflow/cc/saved_model/loader.cc:317


Model downloaded successfully at ./ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Contents of ./ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz extracted to ./ssd_resnet50_v1_fpn_640x640_coco17_tpu-8


W0000 00:00:1716275433.803547      27 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1716275433.803593      27 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-05-21 07:10:33.803795: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model
2024-05-21 07:10:33.874784: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-05-21 07:10:33.874818: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: ./ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model
2024-05-21 07:10:34.562030: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-05-21 07:10:35.707279: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: ./ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model
2024-05-21 07:10:36.238701: I tensorflow/cc/saved_model/loader.cc:317] SavedModel lo

In [32]:
def print_tflite_model_info(model_path):
    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    print("Input details:")
    for detail in input_details:
        print("Name:", detail['name'])
        print("Shape:", detail['shape'])
        print("Type:", detail['dtype'])
        print()

    print("Output details:")
    for detail in output_details:
        print("Name:", detail['name'])
        print("Shape:", detail['shape'])
        print("Type:", detail['dtype'])
        print()

# Replace 'your_model_path.tflite' with the path to your TFLite model
model_path = '/tf/Bolt/tflite_models/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tflite'
print_tflite_model_info(model_path)


Input details:
Name: serving_default_input_tensor:0
Shape: [1 1 1 3]
Type: <class 'numpy.uint8'>

Output details:
Name: StatefulPartitionedCall:6
Shape: [    1 51150     4]
Type: <class 'numpy.float32'>

Name: StatefulPartitionedCall:0
Shape: [1 1]
Type: <class 'numpy.float32'>

Name: StatefulPartitionedCall:5
Shape: [1]
Type: <class 'numpy.float32'>

Name: StatefulPartitionedCall:7
Shape: [    1 51150    91]
Type: <class 'numpy.float32'>

Name: StatefulPartitionedCall:1
Shape: [1 1 1]
Type: <class 'numpy.float32'>

Name: StatefulPartitionedCall:2
Shape: [1 1]
Type: <class 'numpy.float32'>

Name: StatefulPartitionedCall:4
Shape: [1 1]
Type: <class 'numpy.float32'>

Name: StatefulPartitionedCall:3
Shape: [1 1 1]
Type: <class 'numpy.float32'>



In [40]:
import numpy as np

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output tensor indices
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

# Get input tensor details
input_details = interpreter.get_input_details()[0]

# Check the input type and create a dummy input accordingly
if input_details['dtype'] == np.float32:
    dummy_input = np.zeros(input_details['shape'], dtype=np.float32)
elif input_details['dtype'] == np.uint8:
    # If the input type is uint8, generate random values between 0 and 255
    dummy_input = np.random.randint(0, 256, size=input_details['shape'], dtype=np.uint8)
else:
    raise ValueError("Unsupported input type")

# Set the dummy input tensor
interpreter.set_tensor(input_index, dummy_input)

# Invoke inference to trace execution paths
interpreter.invoke()
n = len(interpreter.get_tensor_details())
# Find tensors that are used during inference
used_tensors = set(interpreter.tensor(i)() for i in range(n))

# Filter out unused tensors and save the optimized model
optimized_tensors = [tensor for tensor in interpreter.tensor_details if tensor['index'] in used_tensors]
interpreter.resize_tensor_input(input_index, [1] + list(input_details['shape'][1:]))
interpreter.resize_tensor_input(output_index, [1] + list(interpreter.get_output_details()[0]['shape'][1:]))
interpreter.allocate_tensors()

for idx, tensor in enumerate(optimized_tensors):
    interpreter.set_tensor(idx, interpreter.tensor(tensor['index'])())

# Save the optimized model
with open(model_path+"_optimized_model.tflite", "wb") as f:
    f.write(interpreter.tensor_details)


TypeError: unhashable type: 'numpy.ndarray'